In [ ]:
!pip install pandas langdetect googletrans==4.0.0-rc1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=6c06a97843a885afb81edee91f6358cdce5c142657a07719143150b49da6e63e
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
  Created wheel fo

In [ ]:
!pip install google-cloud-translate


In [ ]:
!pip install deepl

In [ ]:
!pip install prophet


Challenge-1

In [ ]:
import deepl

# Replace 'your_api_key' with your actual DeepL API key
translator = deepl.Translator("your_api_key")

def translate_text(text, target_lang="EN"):
    if pd.isnull(text):
        return np.nan
    result = translator.translate_text(text, target_lang=target_lang)
    return result.text.lower()

# Continue with your preprocessing logic here...


In [ ]:
import pandas as pd
from langdetect import detect
from googletrans import Translator
import re
import numpy as np

# Initialize the Google Translate API client
translator = Translator()

def translate_text(text):
    try:
        if pd.isnull(text):
            return np.nan

        # Detect the language
        detected_language = detect(text)

        # Translate to English if not already in English
        if detected_language != 'en':
            translation = translator.translate(text, src=detected_language, dest='en')
            text = translation.text
        return text.lower()
    except Exception as e:
        print(f"Error during translation: {e}")
        return text.lower()

def normalize_text(text):
    if pd.isnull(text):
        return np.nan
    # Convert to lowercase and remove special characters (retain spaces)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

def preprocess_multilingual_orders(df):
    df['product'] = df['product'].apply(translate_text).apply(normalize_text)
    df['special_instructions'] = df['special_instructions'].apply(translate_text).apply(normalize_text)
    df['special_instructions'] = df['special_instructions'].fillna('no instructions')
    return df

# Sample Data
data = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product': ['T-shirt', 'জিন্স', '衬衫'],
    'quantity': [1000, 500, 750],
    'special_instructions': ['Rush order', 'দ্রুত প্রয়োজন', 'Standard shipping']
})

# Preprocess Data
preprocessed_data = preprocess_multilingual_orders(data)
print(preprocessed_data)


   order_id product  quantity special_instructions
0         1  tshirt      1000           rush order
1         2   jeans       500          fast needed
2         3               750    standard shipping


Challenge-2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CHD-MANSURA/LLM/production_data.csv')

# Preprocess the data
# Convert 'Date' column to datetime format and ensure 'Production Output' is numeric
df['Date'] = pd.to_datetime(df['Date'])
df['y'] = pd.to_numeric(df['Production Output'], errors='coerce')

# Fill or handle missing data (forward fill in this case)
df['y'] = df['y'].fillna(method='ffill')

# Rename 'Date' to 'ds' as required by Prophet
df.rename(columns={'Date': 'ds'}, inplace=True)

# Handle external regressors: 'Worker Attendance' and 'Power Availability Hours'
df['Worker Attendance'] = df['Worker Attendance'].fillna(df['Worker Attendance'].mean())
df['Power Availability Hours'] = df['Power Availability Hours'].fillna(df['Power Availability Hours'].mean())

# Handle the 'Holiday' column
df['Holiday'] = df['Holiday'].fillna(0)

# Initialize the Prophet model and add external regressors
model = Prophet()

# Add external regressors
model.add_regressor('Worker Attendance')
model.add_regressor('Power Availability Hours')
model.add_regressor('Holiday')

# Fit the model
model.fit(df)

# Make future dataframe for the next 30 days
future = model.make_future_dataframe(periods=30)

# Add the regressors to the future dataframe
# For simplicity, use mean values for future regressors
future['Worker Attendance'] = np.random.normal(df['Worker Attendance'].mean(), df['Worker Attendance'].std(), len(future))
future['Power Availability Hours'] = np.random.normal(df['Power Availability Hours'].mean(), df['Power Availability Hours'].std(), len(future))
future['Holiday'] = np.random.choice([0, 1], size=len(future), p=[0.9, 0.1])  # Random example for holiday

# Make predictions
forecast = model.predict(future)

# Show the predictions including uncertainty intervals
forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30)

# Display the results
print(forecast_tail)

# Function to get predictions for the next 30 days
def get_forecast(data_path, future_periods=30):
    df = pd.read_csv(data_path)

    # Preprocess the data
    df['Date'] = pd.to_datetime(df['Date'])
    df['y'] = pd.to_numeric(df['Production Output'], errors='coerce')
    df['y'] = df['y'].fillna(method='ffill')
    df.rename(columns={'Date': 'ds'}, inplace=True)
    df['Worker Attendance'] = df['Worker Attendance'].fillna(df['Worker Attendance'].mean())
    df['Power Availability Hours'] = df['Power Availability Hours'].fillna(df['Power Availability Hours'].mean())
    df['Holiday'] = df['Holiday'].fillna(0)

    # Initialize the Prophet model
    model = Prophet()
    model.add_regressor('Worker Attendance')
    model.add_regressor('Power Availability Hours')
    model.add_regressor('Holiday')

    # Fit the model
    model.fit(df)

    # Create future dataframe and add regressors
    future = model.make_future_dataframe(periods=future_periods)
    future['Worker Attendance'] = np.random.normal(df['Worker Attendance'].mean(), df['Worker Attendance'].std(), len(future))
    future['Power Availability Hours'] = np.random.normal(df['Power Availability Hours'].mean(), df['Power Availability Hours'].std(), len(future))
    future['Holiday'] = np.random.choice([0, 1], size=len(future), p=[0.9, 0.1])

    # Make predictions
    forecast = model.predict(future)

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(future_periods)

# Example usage of the function
forecast_results = get_forecast('/content/drive/MyDrive/CHD-MANSURA/LLM/production_data.csv')
print(forecast_results)


<ipython-input-8-f6a7b11eb480>:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='ffill')
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcmz9tvft/dqbhc725.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcmz9tvft/q6bxxxzj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=69146', 'data', 'file=/tmp/tmpcmz9tvft/dqbhc725.json', 'init=/tmp/tmpcmz9tvft/q6bxxxzj.json', 'output', 'file=/tmp/tmpcmz9tvft/prophet_modelagv6gha0/prophet_model-20240907023208.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:32:08 - cmdstanpy - IN

            ds         yhat  yhat_lower   yhat_upper
365 2024-01-01   967.431803  824.995020  1108.587106
366 2024-01-02   987.484729  832.082274  1115.543693
367 2024-01-03   997.144584  849.214110  1142.302592
368 2024-01-04   988.562049  851.282634  1124.392268
369 2024-01-05   966.456453  816.120048  1110.291239
370 2024-01-06   981.593321  835.542434  1118.918697
371 2024-01-07   979.014812  827.139657  1115.962429
372 2024-01-08   975.126807  823.854576  1121.263770
373 2024-01-09   980.119318  848.687394  1138.107728
374 2024-01-10   998.356086  854.911775  1137.082878
375 2024-01-11   992.756870  851.918776  1131.201763
376 2024-01-12   991.605504  851.267444  1127.858635
377 2024-01-13   988.402998  835.326757  1134.030411
378 2024-01-14   998.666339  861.653676  1145.027469
379 2024-01-15   947.318839  809.221431  1101.758370
380 2024-01-16   974.060437  828.599802  1120.609090
381 2024-01-17   970.269253  835.120551  1117.424523
382 2024-01-18   974.454473  840.275389  1127.

Challenge-3

In [ ]:
class QualityControlAgent:
    def __init__(self, length_threshold=24.0, width_threshold=16.0, sleeve_length_threshold=21.0,
                 defect_rate_threshold=0.01, escalation_defect_rate_threshold=0.03):
        # Initialize thresholds and other parameters
        self.length_threshold = length_threshold
        self.width_threshold = width_threshold
        self.sleeve_length_threshold = sleeve_length_threshold
        self.defect_rate_threshold = defect_rate_threshold
        self.escalation_defect_rate_threshold = escalation_defect_rate_threshold
        self.log = []

    def evaluate_batch(self, batch_data):
        # Extract data from the input
        measurements = batch_data.get('measurements', {})
        defect_rate = batch_data.get('defect_rate', 0)
        batch_size = batch_data.get('batch_size', 0)

        # Check measurements
        length_ok = measurements.get('length', 0) >= self.length_threshold
        width_ok = measurements.get('width', 0) >= self.width_threshold
        sleeve_length_ok = measurements.get('sleeve_length', 0) >= self.sleeve_length_threshold

        # Make decision based on measurements and defect rate
        if length_ok and width_ok and sleeve_length_ok:
            if defect_rate <= self.defect_rate_threshold:
                decision = 'Approve'
                reason = 'All measurements and defect rate are within acceptable ranges.'
            elif defect_rate <= self.escalation_defect_rate_threshold:
                decision = 'Escalate to Supervisor'
                reason = 'Defect rate is slightly above the acceptable threshold.'
            else:
                decision = 'Reject'
                reason = 'Defect rate is above the acceptable threshold.'
        else:
            decision = 'Reject'
            reason = 'One or more measurements are below acceptable thresholds.'

        # Log the decision and reason
        self.log.append({
            'batch_size': batch_size,
            'decision': decision,
            'reason': reason
        })

        return decision, reason

    def get_log(self):
        return self.log

# Example usage
batch_data = {
    'measurements': {
        'length': 25.2,
        'width': 15.8,
        'sleeve_length': 22.1
    },
    'defect_rate': 0.02,
    'batch_size': 1000
}

# Create an instance of the QualityControlAgent
qc_agent = QualityControlAgent()

# Evaluate the batch
decision, reason = qc_agent.evaluate_batch(batch_data)
print(f"Decision: {decision}")
print(f"Reason: {reason}")

# Print the log
print("Log:")
print(qc_agent.get_log())


Decision: Reject
Reason: One or more measurements are below acceptable thresholds.
Log:
[{'batch_size': 1000, 'decision': 'Reject', 'reason': 'One or more measurements are below acceptable thresholds.'}]


Challenge-4

In [ ]:
# Install required packages
!pip install transformers datasets torch

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric
import torch
from sklearn.model_selection import train_test_split

# Load and prepare the dataset
data = pd.read_csv('/content/drive/MyDrive/CHD-MANSURA/LLM/labor_law_compliance_dataset.csv')

# Verify data
print(data.head())

# Ensure correct column names
data.rename(columns={'text': 'text', 'label': 'labels'}, inplace=True)

# Split the dataset
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenizer and model setup
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, return_tensors="pt")

# Preprocess the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define a metric to evaluate the model
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize Trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_train_dataset,    # training dataset
    eval_dataset=tokenized_val_dataset,     # evaluation dataset
    compute_metrics=compute_metrics      # evaluation metrics
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

# Function to classify new text
def classify_text(texts):
    model = BertForSequenceClassification.from_pretrained('./fine_tuned_model')
    tokenizer = BertTokenizer.from_pretrained('./fine_tuned_model')
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage of the function
new_texts = [
    "All workers are entitled to a minimum of 10 days of paid vacation annually.",
    "Workers are required to work 12-hour shifts without any overtime pay."
]
predictions = classify_text(new_texts)
print(predictions)  # Outputs the model's predictions


                                                text  label
0  All workers are entitled to a minimum of 10 da...      1
1  Overtime is not compensated as per the legal r...      0
2  Child labor is strictly prohibited in all comp...      1
3  Workers are required to work 12-hour shifts wi...      0
4  Health and safety regulations are strictly fol...      1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<ipython-input-4-94de45c3d129>:52: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Step,Training Loss


tensor([1, 1])
